In [ ]:
import pandas as pd
import numpy as np
from pandas import json_normalize
import json

# Movie Metadata 

In [ ]:
movie_data = pd.read_csv("movie_data/movies_metadata.csv")
movie_data.head()

In [ ]:
movie_data.shape

In [ ]:
movie_data.columns

In [ ]:
movie_data.dtypes

In [ ]:
# convert budget col to number

# get rid of rows that contain letters first 
bad_budget_rows = movie_data["budget"].str.isnumeric()
movie_data = movie_data[bad_budget_rows]
movie_data["budget"] = movie_data["budget"].astype(np.float64)
movie_data = movie_data[(movie_data["revenue"] != 0) & (movie_data["budget"] != 0) & (movie_data["runtime"] !=0)].reset_index(drop=True)

In [ ]:
# columns that are already cleaned
movie_df = movie_data[["id", "revenue", "vote_average", "title", "budget", "runtime", "original_language"]].copy()
movie_df.shape

In [ ]:
# JSON cols 

# belongs to collection -> boolean
# genres -> top 1
# production companies -> top 1
# production countries -> top 1 
# release date -> extract month and year 

In [ ]:
# Franchise
movie_df["is_franchise"] = ~movie_data["belongs_to_collection"].isna()

In [ ]:
# genres 
def get_json_val(x):
    x_dict = eval(x)
    return np.nan if len(x_dict) == 0 else x_dict[0]["name"]

movie_df["genre"] = movie_data["genres"].apply(get_json_val)
movie_df.dropna(inplace=True)
movie_df.shape

In [ ]:
# production company 
movie_df["production_company"] = movie_data["production_companies"].apply(get_json_val)
movie_df.dropna(inplace=True)
movie_df.shape

In [ ]:
# country
movie_df["country"] = movie_data["production_countries"].apply(get_json_val)
movie_df.dropna(inplace=True)
movie_df.shape

In [ ]:
# release date 
dates = pd.to_datetime(movie_data["release_date"])
movie_df["release_month"] = dates.dt.month.astype("category")
movie_df["release_year"] = dates.dt.year.astype("category")
movie_df.dropna(inplace=True)
movie_df.head()

In [ ]:
# formatted = pd.get_dummies(movie_df.set_index("id").drop("title", axis=1))

# Credits data

In [ ]:
credits = pd.read_csv("movie_data/credits.csv")
credits.head()

In [ ]:
def get_actors(x):
    x_dict = eval(x)
    x_len = len(x_dict)
    cast = []
    for i in range(min(x_len, 3)):
        cast.append(x_dict[i]["name"])
    return cast

In [ ]:
credits["actors"] = credits["cast"].apply(get_actors)
credits.head()

In [ ]:
credits = credits.join(pd.DataFrame(credits["actors"].to_list(), columns=['actor_1', 'actor_2', 'actor_3']))
credits.drop(["actors", "cast"], axis=1, inplace=True)

In [ ]:
credits.head()

In [ ]:
def get_crew(x, dep):
    x_dict = eval(x)
    for pos in x_dict:
        if pos["department"] == dep:
            return pos["name"]
    return "None"

In [ ]:
credits["Director"] = credits["crew"].apply(get_crew, dep="Directing")
credits["Producer"] = credits["crew"].apply(get_crew, dep="Production")
credits["Screenwriter"] = credits["crew"].apply(get_crew, dep="Writing")

In [ ]:
credits.drop("crew", axis=1, inplace=True)
credits.head()

# Keywords data

In [ ]:
keywords_df = pd.read_csv("movie_data/keywords.csv", index_col = "id")
ids = set(movie_df["id"].astype(int)).intersection(keywords_df.index)
keywords_df = keywords_df.loc[ids]
keywords_df.head()

In [ ]:
def get_keywords(x):
    x_dict = eval(x)
    return [keyword["name"] for keyword in x_dict]

In [ ]:
keywords = keywords_df["keywords"].apply(get_keywords)
keywords[:5]

In [ ]:
keywords_df["keyword"] = ["None" if len(row) == 0 else row[0] for row in keywords]
keywords_df.drop(['keywords'], axis=1, inplace=True)
keywords_df.head()

# Joins